In [2]:
import torch
import os
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
from PIL import Image 

In [3]:
import torch.nn as nn
model = models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
# model.load_state_dict(torch.load('flame2resnet.pth'))
model.load_state_dict(torch.load('ff-det_resnet50.pth'))
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/local_scratch/slurm.1310221/ipykernel_1732048/3928000941.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` wil

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
transform = transforms.Compose([
    transforms.Resize((254, 254)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset_path = '../BoWFireDataset/dataset/img'

class_labels = {0: 'not_fire', 1: 'fire'}

In [5]:
correct = 0
total = 0

for img_name in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, img_name)
    
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')) or os.path.isdir(img_path):
        continue
        
    try:
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)
        
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_label = class_labels[predicted.item()]
        
        true_label = 'fire' if 'fire' in img_name.lower() else 'not_fire'
        
        if predicted_label == true_label:
            correct += 1
        total += 1
    
    except Exception as e:
        print(f"Error processing image {img_name}: {e}")

accuracy = (correct / total) * 100 if total > 0 else 0
print(f"Model Accuracy: {accuracy:.2f}%")

Model Accuracy: 27.43%


In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score

correct = 0
total = 0
all_preds = []
all_labels = []

for img_name in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, img_name)
    
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')) or os.path.isdir(img_path):
        continue
        
    try:
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)
        
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_label = class_labels[predicted.item()]
        
        true_label = 'fire' if 'fire' in img_name.lower() else 'not_fire'
        
        # Append predictions and true labels for metric computation
        all_preds.append(predicted_label)
        all_labels.append(true_label)
        
        if predicted_label == true_label:
            correct += 1
        total += 1
    
    except Exception as e:
        print(f"Error processing image {img_name}: {e}")

# Calculate accuracy
accuracy = (correct / total) * 100 if total > 0 else 0

# Calculate Precision, Recall, and F1 Score
precision = precision_score(all_labels, all_preds, pos_label='fire', average='binary')
recall = recall_score(all_labels, all_preds, pos_label='fire', average='binary')
f1 = f1_score(all_labels, all_preds, pos_label='fire', average='binary')

# Print results
print(f"Model Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Model Accuracy: 27.43%
Precision: 1.00
Recall: 0.27
F1 Score: 0.43


In [1]:
import os

def count_jpg_files(directory):
    # Count files ending with .jpg or .jpeg (case insensitive)
    jpg_count = len([f for f in os.listdir(directory) 
                    if f.lower().endswith(('.png','.jpg', '.jpeg'))])
    return jpg_count

directory = "BoWFireDataset/dataset/img"
count = count_jpg_files(directory)
print(f"Found {count} JPG files")

Found 226 JPG files
